In [29]:

# import .py files
import os
import pandas as pd
# import train_model
# import generate_synthetic
# import train_classifier
# import test_imgs
import split_dataset as split
import train_model as train

In [10]:
path_home = "/Users/hadare/Documents/CodingProjects/SyntheticEvaluation"
path_img =  path_home + "/images"
path_dataset = path_home + "/dataset"
path_exp = path_home + "/experiments"
path_models = path_home + "/models"
raw_data = path_home + "/data/mnist_images"

In [3]:
# set parameters for which train_size, gen_size, synthetic/real ratio, GAN train cutoff
# set initial dir for experiments

# Set parameters as lists of integer values
seed = 42
train_sizes = [1000, 2000, 3000]  # List of different training sizes
gen_sizes = [500, 1000, 1500]      # List of different generation sizes
synthetic_real_ratio = 0.5          # Ratio of synthetic to real data
gan_train_cutoff = 5000             # Number of GAN training iterations before switching
train_ratio = 0.8


# Print out the configured parameters for verification
print("Training Configuration:")
print(f"Train Size: {train_sizes}")
print(f"Generation Size: {gen_sizes}")
print(f"Synthetic/Real Ratio: {synthetic_real_ratio}")
print(f"StyleGAN2-ADA Training Cutoff: {gan_train_cutoff}")
# print(f"Experiment Run Directory: {experiment_run_dir}")
print("-" * 40)  # Separator for clarity


Training Configuration:
Train Size: [1000, 2000, 3000]
Generation Size: [500, 1000, 1500]
Synthetic/Real Ratio: 0.5
StyleGAN2-ADA Training Cutoff: 5000
----------------------------------------


In [4]:
# Use preprocessing.ipynb to create a proper dataset
# Distribute files to relevant subfolders + create JSON


In [20]:
# split into train/test

full_json = path_home + '/data/dataset_full.json'

# split into test/train subsets
input_file = full_json



train_df, test_df = split.split_train_test(input_file, train_ratio, seed=seed)

train_output_file = f"{raw_data}/train_data.json"
test_output_file = f"{raw_data}/test_data.json"

split.save_data(train_df, train_output_file)
split.save_data(test_df, test_output_file)

split.print_class_distribution(train_df, "Train")
split.print_class_distribution(test_df, "Test")


Splitting data: 100%|██████████| 10/10 [00:00<00:00, 166.25class/s]

Train data statistics:
Total number of samples: 47995
label
0    4738
1    5393
2    4766
3    4904
4    4673
5    4336
6    4734
7    5012
8    4680
9    4759
Name: count, dtype: int64
Test data statistics:
Total number of samples: 12005
label
0    1185
1    1349
2    1192
3    1227
4    1169
5    1085
6    1184
7    1253
8    1171
9    1190
Name: count, dtype: int64


In [5]:
# train MLP/CNN classifier, test for benchmark using test_imgs.py
# V

In [30]:
# for loop: create dir, create dataset, train model, generate syn-images, test

input_file = raw_data + '/train_data.json'

train_sizes = [1000]
for subset_size in train_sizes:
    # Create a name for the subset and the model
    model_name = f"model_{subset_size // 1000}K"
    # subset the data according to size
    model_dir = path_models + '/' + model_name
    experiments_dir = os.path.join(model_dir, 'experiments')
    dataset_dir = os.path.join(model_dir, 'dataset')
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(experiments_dir, exist_ok=True)


    subset_df = split.subset_data(input_file, subset_size, seed=seed)
    subset_json_path = f"{model_dir}/dataset_subset_size_{subset_size}_seed_{seed}.json"
    split.save_data(subset_df, subset_json_path)
    split.save_data(subset_df, raw_data + "/dataset.json")
    # split.print_class_distribution(subset_df, "Subset")
    
    print(f"Creating dataset for {model_name}...")
    train.create_dataset(path_home, model_dir + '/dataset', raw_data)

    # print(path_home, experiments_dir, dataset_dir)
    # print(f"Training {model_name}...")
    # train.run_stylegan_training(path_home, experiments_dir, dataset_dir, snap=10)



Subsetting classes: 100%|██████████| 10/10 [00:00<00:00, 2269.03class/s]

Creating dataset for model_1K...


AttributeError: module 'train_model' has no attribute 'create_dataset'

In [ ]:
# show results - relevant metric: avg/class accuracy, f1, precision, recall, AUC-ROC...
# graph/tabular


In [ ]:
# delete datasets (keep logs)